In [1]:
import os
import sys
import numpy as np
import pandas as pd
import cv2 as cv
from matplotlib.image import imread
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn import model_selection
from os import listdir
from os.path import isfile, join
import tensorflow as tf
from skimage.io import imread
import cv2
import re

C:\Users\Mary\AppData\Local\Programs\Python\Python37\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
## import the pretrained models

import PIL
import PIL.Image
import tensorflow as tf

In [3]:
dir = os.path.dirname
src_path = dir(os.path.abspath(''))

In [4]:
path = src_path + os.sep + "src" +  os.sep + "utils"  

In [5]:
sys.path.append(path)
import folders_tb as fol 

# VGG16

## X train X Test

In [6]:
path_train = src_path + os.sep + "data" +  os.sep + "train"  +  os.sep + '**'  +  os.sep + '**.jpg'
path_train

'c:\\Users\\Mary\\Desktop\\BootCamp\\Python\\MaryC-MezaR\\data_science_bootcamp_2021\\Machine_Learning_project\\data\\train\\**\\**.jpg'

In [7]:
train = fol.make_imag_df(path_train)

In [8]:
train = fol.creat_colum (train)

In [9]:
path_path = src_path + os.sep + "data" + os.sep + "train"  +  os.sep 

In [10]:
train_set = fol.import_imag(path_path, train)

## X train X Test

In [11]:
path_test = src_path + os.sep + "data" +  os.sep + "test"  +  os.sep + '**'  +  os.sep + '**.jpg'
path_test

'c:\\Users\\Mary\\Desktop\\BootCamp\\Python\\MaryC-MezaR\\data_science_bootcamp_2021\\Machine_Learning_project\\data\\test\\**\\**.jpg'

In [12]:
test = fol.make_imag_df(path_test)

In [13]:
test = fol.creat_colum (test)

In [14]:
path_path_2 = src_path + os.sep + "data" + os.sep + "test"  +  os.sep 

In [15]:
test_set = fol.import_imag(path_path_2, test)
test_set

,path_train,name,id,img,path,label,nombre_img
0,test,battery,battery101,battery101.jpg,battery/battery101.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
1,test,battery,battery108,battery108.jpg,battery/battery108.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
2,test,battery,battery122,battery122.jpg,battery/battery122.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
3,test,battery,battery132,battery132.jpg,battery/battery132.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
4,test,battery,battery14,battery14.jpg,battery/battery14.jpg,0,"[[[255, 255, 255], [255, 255, 255], [255, 255,..."
...,...,...,...,...,...,...,...
1198,test,plastic,plastic864,plastic864.jpg,plastic/plastic864.jpg,6,"[[[182, 191, 200], [185, 194, 203], [184, 192,..."
1199,test,plastic,plastic90,plastic90.jpg,plastic/plastic90.jpg,6,"[[[200, 221, 236], [200, 221, 236], [200, 221,..."
1200,test,plastic,plastic93,plastic93.jpg,plastic/plastic93.jpg,6,"[[[216, 203, 201], [215, 202, 200], [214, 201,..."
1201,test,plastic,plastic95,plastic95.jpg,plastic/plastic95.jpg,6,"[[[255, 242, 240], [255, 242, 240], [255, 242,..."


### DEFINIR X & Y

In [16]:
X_train = np.stack(np.array(train_set["nombre_img"]))
y_train = np.array(train_set["label"])
print(X_train.shape)
print(y_train.shape)

(4803, 224, 224, 3)
(4803,)


In [17]:
X_test = np.stack(np.array(test_set["nombre_img"]))
y_test = np.array(test_set["label"])
print(X_test.shape)
print(y_test.shape)

(1203, 224, 224, 3)
(1203,)


## RESHAPE

In [18]:
X_train = X_train.reshape(4803, 224, 224, 3)
X_train.shape

(4803, 224, 224, 3)

In [19]:
y_train = y_train.reshape(4803, 1)
y_train.shape

(4803, 1)

In [20]:
X_val = X_train[-900:]
y_val = y_train[-900:]
X_train = X_train[:-900]
y_train = y_train[:-900]

In [21]:
X_test = X_test.reshape(1203, 224, 224, 3)
y_test = y_test.reshape(1203,1)

In [22]:
datagen = ImageDataGenerator(rescale=1./255)

In [23]:
datagen.fit(X_train)

In [24]:
datagen.fit(X_test)

In [25]:
datagen.fit(X_val)

### ENTRENAR EL MODELO BASICO

In [26]:
base_model = VGG16(input_shape = (224, 224, 3),
                  include_top=False, 
                  weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False 
x0 = layers.Flatten()(base_model.output) 
x1 = layers.Dense(512, activation='relu')(x0) 
x2 = layers.Dropout(0.5)(x1)
x3 = layers.Dense(7, activation='softmax')(x2)
model = tf.keras.models.Model(base_model.input, x3) 

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy',metrics = ['acc'])

In [27]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [28]:
print("Fit model on training data")
history = model.fit(
    X_train,
    y_train,
    epochs=10, verbose=1,
    validation_data=(X_val, y_val),
)

Fit model on training data
Epoch 1/10
122/122 [==============================] - 882s 7s/step - loss: 8.9769 - acc: 0.8219 - val_loss: 68.4768 - val_acc: 0.2078
Epoch 2/10
122/122 [==============================] - 839s 7s/step - loss: 1.4127 - acc: 0.9283 - val_loss: 93.9466 - val_acc: 0.2089
Epoch 3/10
122/122 [==============================] - 834s 7s/step - loss: 1.1306 - acc: 0.9447 - val_loss: 95.3365 - val_acc: 0.2056
Epoch 4/10
122/122 [==============================] - 802s 7s/step - loss: 0.7363 - acc: 0.9618 - val_loss: 88.0542 - val_acc: 0.2056
Epoch 5/10
122/122 [==============================] - 785s 6s/step - loss: 0.7933 - acc: 0.9646 - val_loss: 120.3599 - val_acc: 0.2122
Epoch 6/10
122/122 [==============================] - 809s 7s/step - loss: 0.6700 - acc: 0.9682 - val_loss: 136.9686 - val_acc: 0.2089
Epoch 7/10
122/122 [==============================] - 799s 7s/step - loss: 0.8240 - acc: 0.9662 - val_loss: 125.3274 - val_acc: 0.2133
Epoch 8/10
122/122 [============

In [30]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=1)

38/38 [==============================] - 176s 5s/step - loss: 82.5208 - acc: 0.5195


In [31]:
print('\nTest accuracy:', test_acc, "\nLoss:", test_loss)


Test accuracy: 0.5195344686508179 
Loss: 82.52079010009766


In [34]:
path_save = path = src_path +  os.sep + "models" + os.sep

In [35]:
model.save(path_save + "modelvgg16_v1.h5")